# Census Data for San Antonio Districts

In [1]:
import pandas as pd
import numpy as np
import re

from ccd_functions import read_distict, cal_counts, cal_district_numbers

## District Data

This sheet tells which district each census tract aligns with and what percentage of the district is included. District 0 are census tracts outside of San Antonio.

In [2]:
dist = read_distict()
dist.head()

,council,tract,percent
0,1.0,192000,0.221309
1,1.0,191900,0.000049
2,1.0,192300,0.185373
3,1.0,192100,0.572719
4,1.0,191304,0.786238


## Census Data

This is all the data pulled from the census. 

In [3]:
filename = 'ACSDP5Y2021.DP03-Data.csv'
filename2 = 'ACSDP5Y2021.DP03-Column-Metadata.csv'

In [4]:
dataall = pd.read_csv(filename)
dataall.columns = dataall.columns.str.lower()

dataall = dataall.iloc[1:].reset_index(drop=True)
dataall = dataall.replace('(X)', np.nan).replace('-',np.nan).replace('\*+',np.nan,regex=True).replace('N',np.nan)
# dataall = dataall.dropna(axis=1, how='all')

dataall.geo_id = dataall.geo_id.str[-6:]
dataall.geo_id = dataall.geo_id.astype(str)
dataall = dataall.drop(columns='name')

to_fix = []
for col in dataall.columns:
    bad_num = dataall[col].astype(str).str.contains('+',regex=False).sum()
    if bad_num > 0:
        to_fix.append(col)
for col in to_fix:
    dataall[col] = dataall[col].astype(str)\
    .str.replace('+','',regex=False)\
    .str.replace(',','').astype(float)

dataall.head()

,geo_id,dp03_0001e,dp03_0001m,dp03_0001ma,dp03_0001ea,dp03_0002e,dp03_0002ea,dp03_0002m,dp03_0002ma,dp03_0003e,...,dp03_0135pma,dp03_0136pe,dp03_0136pm,dp03_0136pma,dp03_0136pea,dp03_0137pe,dp03_0137pm,dp03_0137pma,dp03_0137pea,unnamed: 1098
0,110100,2779,583,NaN,NaN,1778,NaN,430,NaN,1715,...,NaN,12.1,19.1,NaN,NaN,21.7,7.8,NaN,NaN,NaN
1,110300,2327,455,NaN,NaN,1718,NaN,433,NaN,1718,...,NaN,5.0,6.8,NaN,NaN,32.7,12.3,NaN,NaN,NaN
2,110500,1376,175,NaN,NaN,679,NaN,128,NaN,662,...,NaN,65.3,8.8,NaN,NaN,73.3,12.8,NaN,NaN,NaN
3,110600,5152,1603,NaN,NaN,939,NaN,216,NaN,939,...,NaN,41.3,19.6,NaN,NaN,50.5,13.8,NaN,NaN,NaN
4,110700,878,204,NaN,NaN,434,NaN,196,NaN,434,...,NaN,25.0,13.5,NaN,NaN,66.0,18.2,NaN,NaN,NaN


## Labels

These are the labels for the census data.

In [5]:
labels = pd.read_csv(filename2)
labels.columns = labels.columns.str.lower().str.replace('\W+','_', regex=True)
labels.column_name = labels.column_name.str.lower()

labels = labels [labels.label.str.startswith('Estimate')]
labels = labels [~labels.label.str.contains('ratio')]

groups = labels.label.str.split('!', expand=True)[2].unique()
groups

array(['EMPLOYMENT STATUS', 'COMMUTING TO WORK', 'OCCUPATION', 'INDUSTRY',
       'CLASS OF WORKER',
       'INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)',
       'HEALTH INSURANCE COVERAGE',
       'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL'],
      dtype=object)

## Now combine and calculate! 

### Labor force

In [6]:
grouping = groups[0]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final1 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP03_0001E')
final1

EMPLOYMENT STATUS

Estimate!!EMPLOYMENT STATUS!!Population 16 years and over
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force
Estimate!!EMPLOYMENT STATUS!!Civilian labor force
Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force
Estimate!!EMPLOYMENT STATUS!!Female

/Users/mists/Projects/sa2020/sa2020-ccd/ccd_functions.py:69: RuntimeWarning: invalid value encountered in double_scalars
  error_perc = round(math.sqrt(sum((subset.tru_error / 1.645)**2)) / count, 2)


,0,1,2,3,4,5,6,7,8,9,10
Population 16 years and over counts,364836.06,112456.09,98558.49,108009.04,97687.66,106861.18,99275.97,121416.35,119582.89,95931.85,111500.41
Population 16 years and over counts percent,0.25,0.08,0.07,0.08,0.07,0.07,0.07,0.08,0.08,0.07,0.08
Population 16 years and over moe,6699.94,3087.09,3065.69,3082.45,2941.94,3396.09,3049.77,3326.69,3047.01,2568.23,2700.20
Population 16 years and over moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01
In labor force counts,245190.22,71078.45,62106.58,63864.51,62342.92,58860.34,69788.80,78710.46,86878.21,67159.57,76326.95
...,...,...,...,...,...,...,...,...,...,...,...
Own children of the householder 6 to 17 years moa percent,0.02,0.05,0.04,0.04,0.04,0.04,0.04,0.05,0.05,0.04,0.04
All parents in family in labor force counts,60028.82,10573.91,14278.87,16342.64,17155.16,14603.31,15865.35,16154.88,11085.15,14244.03,15045.89
All parents in family in labor force counts percent,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
All parents in family in labor force moe,2775.72,1105.66,1322.10,1268.51,1435.47,1208.38,1445.37,1529.60,986.44,1211.00,1337.50


### Commuting

In [7]:
grouping = groups[1]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final2 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP03_0018E')    
final2

COMMUTING TO WORK

Estimate!!COMMUTING TO WORK!!Workers 16 years and over
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- drove alone
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- carpooled
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Public transportation (excluding taxicab)
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Walked
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Other means
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Worked from home
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Mean travel time to work (minutes)


,0,1,2,3,4,5,6,7,8,9,10
Workers 16 years and over counts,229810.16,66130.00,56537.67,59402.88,56994.80,52739.96,63283.90,72496.97,80603.38,63775.17,71678.11
Workers 16 years and over counts percent,0.26,0.08,0.06,0.07,0.07,0.06,0.07,0.08,0.09,0.07,0.08
Workers 16 years and over moe,5298.75,2266.93,2434.79,2167.23,2059.86,2004.09,2225.63,2362.98,2507.65,2009.17,2220.61
Workers 16 years and over moa percent,0.01,0.02,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
"Car, truck, or van -- drove alone counts",172249.19,47916.80,41696.02,43368.03,41237.13,38507.27,48484.73,54510.58,58782.86,49099.22,56149.17
"Car, truck, or van -- drove alone counts percent",0.20,0.05,0.05,0.05,0.05,0.04,0.06,0.06,0.07,0.06,0.06
"Car, truck, or van -- drove alone moe",3883.33,1852.62,2184.81,1806.85,1652.96,1678.86,1928.34,1928.50,2014.20,1784.61,1952.03
"Car, truck, or van -- drove alone moa percent",0.01,0.02,0.03,0.03,0.02,0.03,0.02,0.02,0.02,0.02,0.02
"Car, truck, or van -- carpooled counts",24290.03,7131.49,7306.55,9610.97,9575.94,8619.68,7919.66,6845.43,7224.58,4834.11,7061.55
"Car, truck, or van -- carpooled counts percent",0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


### Occupation

In [8]:
grouping = groups[2]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final3 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP03_0026E')
final3

OCCUPATION

Estimate!!OCCUPATION!!Civilian employed population 16 years and over
Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Management, business, science, and arts occupations
Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Service occupations
Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Sales and office occupations
Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Natural resources, construction, and maintenance occupations
Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Production, transportation, and material moving occupations


,0,1,2,3,4,5,6,7,8,9,10
Civilian employed population 16 years and over counts,222361.98,66975.62,54844.53,60340.82,56817.36,53968.38,64507.10,73601.38,81670.64,64046.33,72015.86
Civilian employed population 16 years and over counts percent,0.26,0.08,0.06,0.07,0.07,0.06,0.07,0.08,0.09,0.07,0.08
Civilian employed population 16 years and over moe,4653.54,2324.53,2398.54,2169.47,2069.19,2022.04,2347.28,2373.69,2564.35,2123.55,2212.75
Civilian employed population 16 years and over moa percent,0.01,0.02,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
"Management, business, science, and arts occupations counts",94370.83,21964.27,13488.95,13829.88,12523.62,9315.68,23543.21,28937.98,38500.03,33028.91,28389.64
"Management, business, science, and arts occupations counts percent",0.11,0.03,0.02,0.02,0.01,0.01,0.03,0.03,0.04,0.04,0.03
"Management, business, science, and arts occupations moe",2618.07,1272.00,1017.89,930.70,873.94,842.46,1390.98,1409.65,1722.55,1379.94,1448.62
"Management, business, science, and arts occupations moa percent",0.02,0.04,0.05,0.04,0.04,0.05,0.04,0.03,0.03,0.03,0.03
Service occupations counts,33873.49,15089.97,13737.86,14574.12,13268.37,14608.70,12856.60,13110.47,13670.05,8699.77,12051.60
Service occupations counts percent,0.04,0.02,0.02,0.02,0.02,0.02,0.01,0.02,0.02,0.01,0.01


### Industry

In [9]:
grouping = groups[3]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final4 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP03_0032E')
final4

INDUSTRY

Estimate!!INDUSTRY!!Civilian employed population 16 years and over
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Construction
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Manufacturing
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Wholesale trade
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Retail trade
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Transportation and warehousing, and utilities
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Information
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Finance and insurance, and real estate and rental and leasing
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Professional, scientific, and management, and administrative and waste managemen

,0,1,2,3,4,5,6,7,8,9,10
Civilian employed population 16 years and over counts,222361.98,66975.62,54844.53,60340.82,56817.36,53968.38,64507.10,73601.38,81670.64,64046.33,72015.86
Civilian employed population 16 years and over counts percent,0.26,0.08,0.06,0.07,0.07,0.06,0.07,0.08,0.09,0.07,0.08
Civilian employed population 16 years and over moe,4653.54,2324.53,2398.54,2169.47,2069.19,2022.04,2347.28,2373.69,2564.35,2123.55,2212.75
Civilian employed population 16 years and over moa percent,0.01,0.02,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
"Agriculture, forestry, fishing and hunting, and mining counts",3607.14,491.78,869.74,764.35,410.69,228.36,598.73,476.04,638.98,530.48,678.71
"Agriculture, forestry, fishing and hunting, and mining counts percent",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Agriculture, forestry, fishing and hunting, and mining moe",641.18,158.20,339.81,257.97,180.03,127.32,255.68,196.03,216.39,181.31,201.71
"Agriculture, forestry, fishing and hunting, and mining moa percent",0.11,0.20,0.24,0.21,0.27,0.34,0.26,0.25,0.21,0.21,0.18
Construction counts,15191.96,7126.95,5240.87,7113.32,7237.94,9152.48,4498.90,5623.06,3381.96,2838.25,5275.30
Construction counts percent,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.01


### Class of worker

In [10]:
grouping = groups[4]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final5 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP03_0046E')
final5

CLASS OF WORKER

Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over
Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over!!Private wage and salary workers
Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over!!Government workers
Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over!!Self-employed in own not incorporated business workers
Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over!!Unpaid family workers


,0,1,2,3,4,5,6,7,8,9,10
Civilian employed population 16 years and over counts,222361.98,66975.62,54844.53,60340.82,56817.36,53968.38,64507.10,73601.38,81670.64,64046.33,72015.86
Civilian employed population 16 years and over counts percent,0.26,0.08,0.06,0.07,0.07,0.06,0.07,0.08,0.09,0.07,0.08
Civilian employed population 16 years and over moe,4653.54,2324.53,2398.54,2169.47,2069.19,2022.04,2347.28,2373.69,2564.35,2123.55,2212.75
Civilian employed population 16 years and over moa percent,0.01,0.02,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Private wage and salary workers counts,171699.20,54550.70,44478.46,47681.36,45482.43,43367.25,51184.65,57810.26,67750.82,51140.40,56874.46
Private wage and salary workers counts percent,0.20,0.06,0.05,0.05,0.05,0.05,0.06,0.07,0.08,0.06,0.07
Private wage and salary workers moe,4232.48,2252.93,2040.56,1971.56,1898.18,1797.83,2192.75,2152.83,2489.57,1883.45,2043.11
Private wage and salary workers moa percent,0.01,0.03,0.03,0.03,0.03,0.03,0.03,0.02,0.02,0.02,0.02
Government workers counts,36676.23,7272.10,6924.97,8544.20,7630.83,6080.69,10181.36,10988.61,10500.20,8688.86,10370.94
Government workers counts percent,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


### Income and benefits

In [11]:
grouping = groups[5][:19]
print(grouping)

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final6 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP03_0051E')
final6

INCOME AND BENEFITS
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!Less than $10,000
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$10,000 to $14,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$15,000 to $24,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$25,000 to $34,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$35,000 to $49,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$50,000 to $74,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$75,000 to $99,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$100,000 to $149,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUS

,0,1,2,3,4,5,6,7,8,9,10
Total households counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Total households counts percent,0.24,0.08,0.07,0.07,0.06,0.07,0.07,0.09,0.09,0.08,0.08
Total households moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Total households moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
"Less than $10,000 counts",4844.33,5061.99,4745.18,5379.88,2477.34,6041.99,1677.76,5035.24,4390.91,1799.27,2411.10
...,...,...,...,...,...,...,...,...,...,...,...
"Median earnings for male full-time, year-round workers (dollars) moa percent",0.02,0.03,0.03,0.03,0.02,0.02,0.03,0.03,0.03,0.03,0.03
"Median earnings for female full-time, year-round workers (dollars) counts",3454193.51,1299895.69,973364.05,994807.49,751143.35,999934.96,849264.03,1269316.17,1253295.30,1278084.18,1231101.29
"Median earnings for female full-time, year-round workers (dollars) counts percent",5.19,1.95,1.46,1.49,1.13,1.50,1.27,1.91,1.88,1.92,1.85
"Median earnings for female full-time, year-round workers (dollars) moe",120994.69,63096.64,59804.36,53440.65,39247.58,42132.65,46153.08,50030.11,53809.77,58335.00,52616.61


### Health Insurance Coverage

In [12]:
grouping = groups[6]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final7 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP03_0095E')
final7

HEALTH INSURANCE COVERAGE

Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With health insurance coverage
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With health insurance coverage!!With private health insurance
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With health insurance coverage!!With public coverage
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!No health insurance coverage
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population under 19 years
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population under 19 years!!No health insurance coverage
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population 19 to 64 years
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population 19 to 64 years!!I

,0,1,2,3,4,5,6,7,8,9,10
Civilian noninstitutionalized population counts,469215.23,135947.01,127336.16,139627.00,131390.02,137979.32,128793.15,151161.86,143873.70,119221.97,139904.59
Civilian noninstitutionalized population counts percent,0.26,0.07,0.07,0.08,0.07,0.08,0.07,0.08,0.08,0.07,0.08
Civilian noninstitutionalized population moe,8069.34,3897.42,4233.04,4025.34,4157.28,4407.51,4404.78,4548.57,3800.39,3347.23,3797.66
Civilian noninstitutionalized population moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
With health insurance coverage counts,412935.20,104828.01,103478.74,111448.71,104688.09,103468.71,110074.78,128392.84,124863.19,107474.03,121567.71
...,...,...,...,...,...,...,...,...,...,...,...
Not in labor force:!!With health insurance coverage!!With public coverage moa percent,0.05,0.06,0.05,0.05,0.07,0.05,0.07,0.07,0.09,0.09,0.07
Not in labor force:!!No health insurance coverage counts,11550.18,7761.89,5768.60,7542.17,5669.47,8997.31,3290.70,5537.97,3366.32,1965.51,3650.87
Not in labor force:!!No health insurance coverage counts percent,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Not in labor force:!!No health insurance coverage moe,1035.78,921.39,769.49,907.76,632.78,799.16,511.71,894.70,581.54,380.09,523.34


### Below poverty level - families

In [13]:
grouping = groups[7]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[:-10]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final8 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP03_0119E')
final8

PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL

Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families!!With related children of the householder under 18 years
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families!!With related children of the householder under 18 years!!With related children of the householder under 5 years only
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families!!Married couple families
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families!!Married couple families!!With related children of the householder under 18 years
Estimate!!PERCENTAGE 

/Users/mists/Projects/sa2020/sa2020-ccd/ccd_functions.py:66: RuntimeWarning: invalid value encountered in double_scalars
  count_perc = round(count / total_counts, 2)


,0,1,2,3,4,5,6,7,8,9,10
All families counts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All families counts percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
All families moe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All families moa percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
With related children of the householder under 18 years counts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
With related children of the householder under 18 years counts percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
With related children of the householder under 18 years moe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
With related children of the householder under 18 years moa percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
With related children of the householder under 18 years!!With related children of the householder under 5 years only counts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
With related children of the householder under 18 years!!With related children of the householder under 5 years only counts percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Below poverty level - families

In [14]:
grouping = groups[7]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[9:]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final9 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP03_0128E')
final9

PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL

Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!Under 18 years
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!Under 18 years!!Related children of the householder under 18 years
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!Under 18 years!!Related children of the householder under 18 years!!Related children of the householder under 5 years
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!Under 18 years!!Related children of the householder under 18 years!!Related children of the householder 5 to 

,0,1,2,3,4,5,6,7,8,9,10
All people counts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All people counts percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
All people moe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All people moa percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Under 18 years counts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Under 18 years counts percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Under 18 years moe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Under 18 years moa percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Under 18 years!!Related children of the householder under 18 years counts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Under 18 years!!Related children of the householder under 18 years counts percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
with pd.ExcelWriter('sa2020_ccd_employment_update.xlsx') as writer:
    final1.to_excel(writer, sheet_name=groups[0][:30])
    final2.to_excel(writer, sheet_name=groups[1][:30])
    final3.to_excel(writer, sheet_name=groups[2][:30])
    final4.to_excel(writer, sheet_name=groups[3][:30])
    final5.to_excel(writer, sheet_name=groups[4][:30])
    final6.to_excel(writer, sheet_name=groups[5][:30])
    final7.to_excel(writer, sheet_name=groups[6][:30])
    final8.to_excel(writer, sheet_name='BELOW POVERTY FAMILY')
    final8.to_excel(writer, sheet_name='BELOW POVERTY PEOPLE')